In [3]:
import pandas as pd

# === 1. Inlezen ===
df = pd.read_excel("Bus_Planning-FIXED.xlsx")
df.columns = df.columns.str.strip().str.lower()

# check of 'duration_minutes' bestaat — anders berekenen
if 'duration_minutes' not in df.columns:
    df['start time'] = pd.to_datetime(df['start time'], errors='coerce')
    df['end time'] = pd.to_datetime(df['end time'], errors='coerce')

    # nacht-rollover fix (als eindtijd < starttijd → +1 dag)
    df['end time'] = df.apply(
        lambda r: r['end time'] + pd.Timedelta(days=1) if pd.notna(r['start time']) and pd.notna(r['end time']) and r['end time'] < r['start time'] else r['end time'],
        axis=1
    )

    df['duration_minutes'] = (df['end time'] - df['start time']).dt.total_seconds() / 60

# === 2. Bereken actieve minuten per bus (zonder idle) ===
eff_df = (
    df[df['activity'].str.lower() != 'idle']
    .groupby('bus', as_index=False)['duration_minutes']
    .sum()
    .rename(columns={'duration_minutes': 'active_minutes'})
)

# === 3. De 3 minst efficiënte bussen ===
least_efficient = eff_df.nsmallest(3, 'active_minutes')['bus'].tolist()
print("De 3 minst efficiënte bussen:", least_efficient)

# === 4. Schrappen ===
filtered_df = df[~df['bus'].isin(least_efficient)].copy()

# === 5. Opslaan ===
filtered_df.to_excel("Busplan_step1_remove_inefficient.xlsx", index=False)

print("Nieuwe busplan opgeslagen als 'Busplan_step1_remove_inefficient.xlsx'")


De 3 minst efficiënte bussen: [20, 19, 18]
Nieuwe busplan opgeslagen als 'Busplan_step1_remove_inefficient.xlsx'


In [9]:
import pandas as pd
import numpy as np

# ============== SETTINGS ==============
SRC_PLAN = "Bus_Planning-FIXED.xlsx"   # input plan
SRC_TT   = "Timetable.xlsx"            # (niet nodig voor loc-matches; wel voor toekomst)
OUT_FILE = "busplan_grouped_plus_charging.xlsx"

TURNOVER_MIN = 4.0                     # min marge (min)
BAT_CAP_KWH        = 300.0
SOC_START_PCT      = 100.0
SOC_TRIGGER_PCT    = 50.0              # laad als SoC < 50%
SOC_TARGET_PCT     = 80.0              # laad tot 80%
CHARGE_RATE_KWHPM  = 2.0               # laadtempo (kWh/min) -> pas aan indien nodig

# ============== HELPERS ==============
def norm(s): return str(s).strip().lower()

def parse_times(df):
    g = df.copy()
    st = pd.to_datetime(g['start time'], format="%H:%M:%S", errors='coerce')
    et = pd.to_datetime(g['end time'],   format="%H:%M:%S", errors='coerce')
    roll = (et < st) & st.notna() & et.notna()
    et = et + pd.to_timedelta(roll.astype(int), unit='D')
    g['_start'] = st
    g['_end']   = et
    g['_dur_min'] = (g['_end'] - g['_start']).dt.total_seconds()/60.0
    return g

def fmt_times(df):
    out = df.copy()
    out['start time'] = pd.to_datetime(out['start time'], errors='coerce').dt.strftime("%H:%M:%S")
    out['end time']   = pd.to_datetime(out['end time'],   errors='coerce').dt.strftime("%H:%M:%S")
    return out

def ensure_cols(d):
    need = ['activity','start location','end location','line','bus','start time','end time','energy consumption']
    miss = [c for c in need if c not in d.columns]
    if miss:
        raise ValueError(f"Missing columns: {miss}")

# ============== LOAD ==============
raw = pd.read_excel(SRC_PLAN, sheet_name="Sheet1")
ensure_cols(raw)
orig_cols = list(raw.columns)

# normaliseer + filter op service trips
df = raw.copy()
df['activity_norm']     = df['activity'].astype(str).str.lower().str.strip()
df['start location_lc'] = df['start location'].astype(str).str.strip().str.lower()
df['end location_lc']   = df['end location'].astype(str).str.strip().str.lower()
df = parse_times(df)

service = df[df['activity_norm'] == 'service trip'].copy().sort_values('_start')

# ============== KETENS MAKEN (geen material trips nodig) ==============
cand = service.copy()
cand['assigned'] = False

chains = []   # (bus_id, [row_indexen in cand])
bus_id = 1

while True:
    unassigned = cand[~cand['assigned']]
    if unassigned.empty:
        break

    chain_idxs = []
    cur_idx = unassigned.index[0]
    cand.loc[cur_idx, 'assigned'] = True
    chain_idxs.append(cur_idx)
    cur_end_time = cand.loc[cur_idx, '_end']
    cur_end_loc  = cand.loc[cur_idx, 'end location_lc']

    while True:
        # pak volgende trip die start waar de vorige eindigde, en pas NA 4 min turnover
        nxt = cand[
            (~cand['assigned']) &
            (cand['start location_lc'] == cur_end_loc) &
            (cand['_start'] >= (cur_end_time + pd.Timedelta(minutes=TURNOVER_MIN)))
        ].sort_values('_start')

        if nxt.empty:
            break

        next_idx = nxt.index[0]
        cand.loc[next_idx, 'assigned'] = True
        chain_idxs.append(next_idx)
        cur_end_time = cand.loc[next_idx, '_end']
        cur_end_loc  = cand.loc[next_idx, 'end location_lc']

    chains.append((bus_id, chain_idxs))
    bus_id += 1

# ============== BASIS-OUTPUT (alleen service, bussen = ketens) ==============
rows = []
for b_id, idxs in chains:
    for ix in idxs:
        r = cand.loc[ix]
        outrow = raw.iloc[r.name].copy()  # exact originele row als basis
        outrow['bus'] = b_id
        rows.append(outrow)

plan = pd.DataFrame(rows)
plan = parse_times(plan).sort_values(['bus','_start','_end']).reset_index(drop=True)

# ============== CHARGING INPLANNEN (SoC<50% → laden tot 80%, buffers enforced) ==============
charged_rows = []
for bus, g in plan.groupby('bus', dropna=False):
    g = g.sort_values('_start').reset_index(drop=True).copy()
    # battery state
    batt = BAT_CAP_KWH * (SOC_START_PCT/100.0)

    i = 0
    while i < len(g):
        r = g.loc[i].copy()
        act = str(r['activity']).lower()
        e = pd.to_numeric(r.get('energy consumption', 0.0), errors='coerce')
        if np.isnan(e): e = 0.0

        # verbruik toepassen
        if 'charging' in act:
            batt = min(BAT_CAP_KWH, batt + abs(e))
        else:
            batt = batt - max(0.0, e)

        charged_rows.append(r)

        # check charge trigger vóór volgende service
        if i < len(g)-1 and batt < BAT_CAP_KWH * (SOC_TRIGGER_PCT/100.0):
            need = BAT_CAP_KWH*(SOC_TARGET_PCT/100.0) - batt
            need = max(0.0, need)

            if need > 0 and CHARGE_RATE_KWHPM > 0:
                ch_min = int(np.ceil(need / CHARGE_RATE_KWHPM))

                # 4 min buffer na current end
                ch_start = r['_end'] + pd.Timedelta(minutes=TURNOVER_MIN)
                ch_end   = ch_start + pd.Timedelta(minutes=ch_min)

                # enforce 4 min buffer vóór volgende service
                target_next_start = ch_end + pd.Timedelta(minutes=TURNOVER_MIN)
                delta = target_next_start - g.loc[i+1, '_start']
                if delta > pd.Timedelta(0):
                    # schuif alle volgende blokken op deze bus
                    g.loc[i+1:, '_start'] += delta
                    g.loc[i+1:, '_end']   += delta

                # voeg charging-row toe (op dezelfde locatie als einde van r)
                ch = r.copy()
                ch['activity']       = 'charging'
                ch['line']           = np.nan
                ch['start location'] = r['end location']
                ch['end location']   = r['end location']
                ch['_start']         = ch_start
                ch['_end']           = ch_end
                ch['start time']     = ch_start
                ch['end time']       = ch_end
                ch['energy consumption'] = -need  # negatief = energie toegevoegd

                charged_rows.append(ch)
                batt = min(BAT_CAP_KWH, batt + need)

        i += 1

# ============== FINAL DF ==============
final = pd.DataFrame(charged_rows).sort_values(['bus','_start','_end']).reset_index(drop=True)
# zet tijden terug naar strings en kolomvolgorde bewaren
final['start time'] = final['_start']
final['end time']   = final['_end']
final = final.drop(columns=['_start','_end','_dur_min'], errors='ignore')

for c in orig_cols:
    if c not in final.columns:
        final[c] = np.nan
final = final[orig_cols]

final = fmt_times(final)
final.to_excel(OUT_FILE, sheet_name="Sheet1", index=False)

print(f"✅ Nieuw plan incl. charging → {OUT_FILE}")
print("🚌 Bussen:", final['bus'].nunique(), "| Rijen:", len(final))


✅ Nieuw plan incl. charging → busplan_grouped_plus_charging.xlsx
🚌 Bussen: 15 | Rijen: 349


In [11]:
import pandas as pd
import numpy as np

# ====== SETTINGS ======
IN_PLAN   = "busplan_grouped_plus_charging.xlsx"  # je huidige plan (service + charging)
TIMETABLE = "Timetable.xlsx"                      # origins in rows, destinations in cols (allemaal lowercase headers)
OUT_FILE  = "busplan_with_materials.xlsx"
TURNOVER_MIN = 4.0                                # 4 min buffer vóór/na material
MAT_KWH_PER_MIN = 10.32 / 20.0                    # 0.516 kWh/min

# ====== HELPERS ======
def ensure_cols(d):
    need = ['activity','start location','end location','line','bus','start time','end time','energy consumption']
    miss = [c for c in need if c not in d.columns]
    if miss:
        raise ValueError(f"Missing columns: {miss}")

def parse_times(df):
    g = df.copy()
    st = pd.to_datetime(g['start time'], format="%H:%M:%S", errors='coerce')
    et = pd.to_datetime(g['end time'],   format="%H:%M:%S", errors='coerce')
    roll = (et < st) & st.notna() & et.notna()
    et = et + pd.to_timedelta(roll.astype(int), unit='D')
    g['_start'] = st
    g['_end']   = et
    return g

def fmt_times(df):
    out = df.copy()
    out['start time'] = pd.to_datetime(out['start time'], errors='coerce').dt.strftime("%H:%M:%S")
    out['end time']   = pd.to_datetime(out['end time'],   errors='coerce').dt.strftime("%H:%M:%S")
    return out

def norm(s): return str(s).strip().lower()

def load_tt(path):
    tt = pd.read_excel(path, index_col=0)
    tt.index   = tt.index.astype(str).str.strip().str.lower()
    tt.columns = tt.columns.astype(str).str.strip().str.lower()
    return tt

def tt_minutes(tt, a, b):
    if pd.isna(a) or pd.isna(b): return None
    a, b = norm(a), norm(b)
    try:
        v = float(tt.loc[a, b])
        return v if np.isfinite(v) else None
    except Exception:
        return None

# ====== LOAD ======
plan = pd.read_excel(IN_PLAN, sheet_name="Sheet1")
ensure_cols(plan)
orig_cols = list(plan.columns)

tt = load_tt(TIMETABLE)

# normalise & sort
plan['activity'] = plan['activity'].astype(str).str.strip()
plan['_sl'] = plan['start location'].astype(str).str.strip().str.lower()
plan['_el'] = plan['end location'].astype(str).str.strip().str.lower()

plan = parse_times(plan)
plan = plan.sort_values(['bus','_start','_end']).reset_index(drop=True)

# ====== INSERT MATERIAL TRIPS ======
rows = []
for bus, g in plan.groupby('bus', dropna=False):
    g = g.sort_values('_start').reset_index(drop=True).copy()

    i = 0
    while i < len(g):
        cur = g.loc[i].copy()
        rows.append(cur)

        if i == len(g) - 1:
            i += 1
            continue

        nxt = g.loc[i+1].copy()

        # als end_loc == next start_loc → geen material nodig
        if str(cur['_el']).lower() == str(nxt['_sl']).lower():
            i += 1
            continue

        # anders: bepaal required travel via TT
        req_min = tt_minutes(tt, cur['end location'], nxt['start location'])
        if req_min is None:    # geen tabelwaarde → we doen niks (veiliger dan gokken)
            i += 1
            continue

        # ideal timing van material trip: start na 4 min buffer, duur = req_min
        mat_start = cur['_end'] + pd.Timedelta(minutes=TURNOVER_MIN)
        mat_end   = mat_start + pd.Timedelta(minutes=req_min)

        # enforce 4-min buffer vóór doorstart naar next
        target_next_start = mat_end + pd.Timedelta(minutes=TURNOVER_MIN)

        # als next te vroeg is, schuif next en alles erna
        if nxt['_start'] < target_next_start:
            delta = target_next_start - nxt['_start']
            g.loc[i+1:, '_start'] += delta
            g.loc[i+1:, '_end']   += delta

        # bouw de material trip
        mat = cur.copy()
        mat['activity']       = 'material trip'
        mat['line']           = np.nan
        mat['start location'] = cur['end location']
        mat['end location']   = nxt['start location']
        mat['_sl']            = str(cur['_el'])
        mat['_el']            = str(nxt['_sl'])
        mat['_start']         = mat_start
        mat['_end']           = mat_end
        mat['start time']     = mat_start
        mat['end time']       = mat_end
        # energieverbruik lineair met duur
        dur_min = (mat_end - mat_start).total_seconds()/60.0
        mat['energy consumption'] = round(MAT_KWH_PER_MIN * dur_min, 3)

        rows.append(mat)

        i += 1

# ====== BUILD OUTPUT ======
out = pd.DataFrame(rows).sort_values(['bus','_start','_end']).reset_index(drop=True)

# strip helper cols & restore order
out['start time'] = out['_start']
out['end time']   = out['_end']
out = out.drop(columns=['_start','_end','_sl','_el'], errors='ignore')

# make sure all original columns exist (preserve your app schema)
for c in orig_cols:
    if c not in out.columns:
        out[c] = np.nan
out = out[orig_cols]

# format times
out = fmt_times(out)

# ====== SAVE ======
out.to_excel(OUT_FILE, sheet_name="Sheet1", index=False)
print(f"✅ Material trips toegevoegd → {OUT_FILE}")
print("🚌 Bussen:", out['bus'].nunique(), "| Rijen:", len(out))


✅ Material trips toegevoegd → busplan_with_materials.xlsx
🚌 Bussen: 15 | Rijen: 349


In [29]:
import pandas as pd
import numpy as np

# ================== SETTINGS ==================
SRC_PLAN = "busplan_with_materials.xlsx"    # jouw huidige plan
SRC_TT   = "Timetable.xlsx"                     # rows=from, cols=to (lowercase labels)
OUT_FILE = "busplan_all_violations_pivotfixed_fullpass.xlsx"
TURNOVER = 4.0
MAX_PASSES = 20                                 # aantal volledige sweeps over alle bussen

BASE_MIN, BASE_KWH = 20.0, 10.32                # material: 20 min => 10.32 kWh
HUBS = ["ehvbst", "ehvgar", "ehvapt"]           # hubs om missing timetable te repareren

# ================== HELPERS ==================
def lc(s): return str(s).strip().lower()

def parse_times(df):
    st = pd.to_datetime(df['start time'], format="%H:%M:%S", errors='coerce')
    et = pd.to_datetime(df['end time'],   format="%H:%M:%S", errors='coerce')
    roll = (et < st) & st.notna() & et.notna()
    et = et + pd.to_timedelta(roll.astype(int), unit='D')
    out = df.copy()
    out['_start'] = st; out['_end'] = et
    return out

def load_tt(path):
    tt = pd.read_excel(path, index_col=0)
    tt.index   = tt.index.astype(str).str.strip().str.lower()
    tt.columns = tt.columns.astype(str).str.strip().str.lower()
    return tt

def tt_min(tt, a, b):
    try:
        v = float(tt.loc[lc(a), lc(b)])
        return v if np.isfinite(v) else None
    except Exception:
        return None

def route_time(tt, a, b):
    """Return (total_minutes, segments) with segments = [(from,to,min), ...]
       Try direct A->B, else try A->HUB->B. None if impossible."""
    a, b = lc(a), lc(b)
    if a == b:
        return 0.0, []
    direct = tt_min(tt, a, b)
    if direct is not None:
        return direct, [(a, b, direct)]
    for h in HUBS:
        ab = tt_min(tt, a, h)
        hb = tt_min(tt, h, b)
        if ab is not None and hb is not None:
            return ab + hb, [(a, h, ab), (h, b, hb)]
    return None, None

def energy_material(minutes):
    return round((BASE_KWH/BASE_MIN)*float(minutes), 3)

def gap_min(a_end, b_start):
    return (b_start - a_end).total_seconds()/60.0

# ---------- violation scanner ----------
def list_violations(plan_df, tt, turnover=TURNOVER, skip_idx=None):
    """Return list of violation row indices in scan order; skip set optional."""
    skip_idx = skip_idx or set()
    viols = []
    for bus, g in plan_df.sort_values(['bus','_start']).groupby('bus'):
        prev_end, prev_eloc = None, None
        for i, r in g.iterrows():
            if lc(r['activity']) != 'service trip':
                prev_end, prev_eloc = r['_end'], r['end location']; continue
            if prev_end is None:
                prev_end, prev_eloc = r['_end'], r['end location']; continue

            if lc(prev_eloc) == lc(r['start location']):
                need = turnover
            else:
                total, segs = route_time(tt, prev_eloc, r['start location'])
                if total is None:
                    need = np.inf  # hard violation (missing timetable)
                else:
                    need = total + turnover

            if gap_min(prev_end, r['_start']) < need and i not in skip_idx:
                viols.append(i)
            prev_end, prev_eloc = r['_end'], r['end location']
    return viols

# ---------- try place one trip in another bus (with hub routing) ----------
def try_place_trip(plan_df, tt, viol_idx, turnover=TURNOVER):
    trip = plan_df.loc[viol_idx].copy()
    src_bus = trip['bus']
    buses = sorted(plan_df['bus'].dropna().unique().tolist())

    # prefer later buses first, then wrap
    if src_bus in buses:
        start_pos = buses.index(src_bus)
        order = buses[start_pos+1:] + buses[:start_pos]
    else:
        order = buses

    def fits_in_gap(prev_row, next_row):
        # links
        if prev_row is not None:
            if lc(prev_row['end location']) == lc(trip['start location']):
                need_L = turnover
                segs_L = []
            else:
                total_L, segs_L = route_time(tt, prev_row['end location'], trip['start location'])
                if total_L is None:
                    return False, None, None
                need_L = total_L + turnover
            if gap_min(prev_row['_end'], trip['_start']) < need_L:
                return False, None, None
        else:
            segs_L = []

        # rechts
        if next_row is not None:
            if lc(trip['end location']) == lc(next_row['start location']):
                need_R = turnover
                segs_R = []
            else:
                total_R, segs_R = route_time(tt, trip['end location'], next_row['start location'])
                if total_R is None:
                    return False, None, None
                need_R = total_R + turnover
            if gap_min(trip['_end'], next_row['_start']) < need_R:
                return False, None, None
        else:
            segs_R = []

        return True, segs_L, segs_R

    for tgt_bus in order:
        g = plan_df[plan_df['bus'] == tgt_bus].sort_values('_start').reset_index()

        slots = []
        if g.empty:
            slots.append(('only', None, None))
        else:
            slots.append(('before_first', None, g.iloc[0]))
            for k in range(len(g)-1):
                slots.append(('between', g.iloc[k], g.iloc[k+1]))
            slots.append(('after_last', g.iloc[-1], None))

        for tag, prev_row, next_row in slots:
            ok, segsL, segsR = fits_in_gap(prev_row, next_row)
            if not ok: 
                continue

            # move trip
            plan_df.loc[viol_idx, 'bus'] = tgt_bus
            base = plan_df.loc[viol_idx].copy()

            # material vóór
            if prev_row is not None:
                t0 = prev_row['_end']
                for (a,b,m) in (segsL or []):
                    if m <= 0: continue
                    mat = base.copy()
                    mat['activity'] = 'material trip'
                    mat['line'] = np.nan
                    mat['start location'] = a
                    mat['end location']   = b
                    mat['_start'] = t0
                    mat['_end']   = t0 + pd.Timedelta(minutes=m)
                    mat['energy consumption'] = energy_material(m)
                    plan_df = pd.concat([plan_df, pd.DataFrame([mat])], ignore_index=True)
                    t0 = mat['_end']

            # material ná
            if next_row is not None:
                t1 = base['_end']
                for (a,b,m) in (segsR or []):
                    if m <= 0: continue
                    mat2 = base.copy()
                    mat2['activity'] = 'material trip'
                    mat2['line'] = np.nan
                    mat2['start location'] = a
                    mat2['end location']   = b
                    mat2['_start'] = t1
                    mat2['_end']   = t1 + pd.Timedelta(minutes=m)
                    mat2['energy consumption'] = energy_material(m)
                    plan_df = pd.concat([plan_df, pd.DataFrame([mat2])], ignore_index=True)
                    t1 = mat2['_end']

            return True, plan_df, tgt_bus, tag

    return False, plan_df, None, None

# ================== MAIN ==================
plan0 = pd.read_excel(SRC_PLAN, sheet_name="Sheet1")
tt = load_tt(SRC_TT)

plan = parse_times(plan0.copy())
plan['activity'] = plan['activity'].astype(str)

passes = 0
total_moves = 0
while passes < MAX_PASSES:
    passes += 1
    skipped = set()
    moved_this_pass = 0

    while True:
        viols = list_violations(plan, tt, TURNOVER, skip_idx=skipped)
        if not viols:
            break

        current = viols[0]
        ok, plan, tgt_bus, slot = try_place_trip(plan, tt, current, TURNOVER)
        if ok:
            plan = plan.sort_values(['bus','_start']).reset_index(drop=True)
            moved_this_pass += 1
            total_moves += 1
            # after a successful move, reset skipped so earlier “stuck” ones can re-attempt
            skipped = set()
            print(f"✅ moved violation idx={current} -> bus {tgt_bus} (slot={slot}).")
        else:
            # can't place this one now — skip and try next violation
            skipped.add(current)
            print(f"⏭️  could not place violation idx={current} this pass; skipping for now.")

        # if we’ve skipped everyone, exit inner loop
        if viols and skipped.issuperset(set(viols)):
            break

    print(f"— pass {passes} done: moved {moved_this_pass} trips.")
    if moved_this_pass == 0:
        # nothing moved in this pass -> stable
        break

# write exact input-format
out = plan.copy()
out['start time'] = pd.to_datetime(out['_start']).dt.strftime("%H:%M:%S")
out['end time']   = pd.to_datetime(out['_end']).dt.strftime("%H:%M:%S")
out = out.drop(columns=['_start','_end'], errors='ignore')
out.to_excel(OUT_FILE, sheet_name="Sheet1", index=False)
print(f"💾 Written: {OUT_FILE}")
print(f"📉 Total moves: {total_moves} across {passes} pass(es).")


✅ moved violation idx=1 -> bus 4 (slot=before_first).
⏭️  could not place violation idx=40 this pass; skipping for now.
✅ moved violation idx=41 -> bus 5 (slot=before_first).
⏭️  could not place violation idx=40 this pass; skipping for now.
⏭️  could not place violation idx=41 this pass; skipping for now.
— pass 1 done: moved 2 trips.
⏭️  could not place violation idx=40 this pass; skipping for now.
⏭️  could not place violation idx=41 this pass; skipping for now.
— pass 2 done: moved 0 trips.
💾 Written: busplan_all_violations_pivotfixed_fullpass.xlsx
📉 Total moves: 2 across 2 pass(es).


In [31]:
import pandas as pd
import numpy as np

# --- SETTINGS ---
SRC_PLAN = "busplan_all_violations_pivotfixed_fullpass.xlsx"   # your current file with the 2 remaining violations
SRC_TT   = "Timetable.xlsx"
OUT_FILE = "busplan_patch_missing_timetable_fixed.xlsx"
TURNOVER = 4.0
HUBS = ["ehvgar"]              # hub(s) to allow routing when A->B is missing
BASE_MIN, BASE_KWH = 20.0, 10.32   # material trip energy rule

def lc(s): return str(s).strip().lower()

def parse_times(df):
    st = pd.to_datetime(df['start time'], format="%H:%M:%S", errors='coerce')
    et = pd.to_datetime(df['end time'],   format="%H:%M:%S", errors='coerce')
    roll = (et < st) & st.notna() & et.notna()
    et = et + pd.to_timedelta(roll.astype(int), unit='D')
    out = df.copy()
    out['_start'] = st; out['_end'] = et
    return out

def load_tt(path):
    tt = pd.read_excel(path, index_col=0)
    tt.index   = tt.index.astype(str).str.strip().str.lower()
    tt.columns = tt.columns.astype(str).str.strip().str.lower()
    return tt

def tt_min(tt, a, b):
    try:
        v = float(tt.loc[lc(a), lc(b)])
        return v if np.isfinite(v) else None
    except Exception:
        return None

def route_time(tt, a, b, hubs=HUBS):
    a, b = lc(a), lc(b)
    if a == b: 
        return 0.0, []
    direct = tt_min(tt, a, b)
    if direct is not None:
        return direct, [(a, b, direct)]
    for h in hubs:
        ab = tt_min(tt, a, h)
        hb = tt_min(tt, h, b)
        if ab is not None and hb is not None:
            return ab + hb, [(a, h, ab), (h, b, hb)]
    return None, None

def energy_material(minutes):
    return round((BASE_KWH/BASE_MIN) * float(minutes), 3)

def gap_min(a_end, b_start):
    return (b_start - a_end).total_seconds()/60.0

# ---------- load + prep ----------
plan0 = pd.read_excel(SRC_PLAN, sheet_name="Sheet1")
tt = load_tt(SRC_TT)
plan = parse_times(plan0.copy())
plan['activity'] = plan['activity'].astype(str)

# ---------- find ONLY the "ehvapt -> ehvbst" missing timetable transitions ----------
def find_specific_mt(plan_df):
    hits = []
    for bus, g in plan_df.sort_values(['bus','_start']).groupby('bus'):
        prev_end, prev_eloc = None, None
        prev_idx = None
        for i, r in g.iterrows():
            if lc(r['activity']) != 'service trip':
                prev_end, prev_eloc, prev_idx = r['_end'], r['end location'], i
                continue
            if prev_end is None:
                prev_end, prev_eloc, prev_idx = r['_end'], r['end location'], i
                continue
            a, b = lc(prev_eloc), lc(r['start location'])
            # target pattern: prev ehvapt -> curr ehvbst AND no direct timetable entry
            if a == "ehvapt" and b == "ehvbst" and tt_min(tt, a, b) is None:
                hits.append((i, bus, prev_idx))
            prev_end, prev_eloc, prev_idx = r['_end'], r['end location'], i
    return hits

targets = find_specific_mt(plan)
print(f"found {len(targets)} 'ehvapt→ehvbst' missing-timetable transitions.")

# ---------- try to move each offending trip ----------
moves_done = 0
for trip_idx, src_bus, prev_idx in targets:
    trip = plan.loc[trip_idx]
    trip_start = trip['_start']
    trip_end   = trip['_end']
    trip_sloc  = trip['start location']
    trip_eloc  = trip['end location']

    # search buses to place before a row whose start location is ehvbst (or where end location is ehvbst for previous)
    buses = sorted(plan['bus'].dropna().unique().tolist())
    placed = False

    for tgt_bus in buses:
        g = plan[plan['bus'] == tgt_bus].sort_values('_start').reset_index()

        # build candidate slots (before_first, between, after_last)
        slots = []
        if g.empty:
            slots.append(('only', None, None))
        else:
            slots.append(('before_first', None, g.iloc[0]))
            for k in range(len(g)-1):
                slots.append(('between', g.iloc[k], g.iloc[k+1]))
            slots.append(('after_last', g.iloc[-1], None))

        for tag, prev_row, next_row in slots:
            # require previous end location on tgt bus to be ehvbst (so no missing timetable),
            # or if prev_row is None but next_row starts at ehvbst we place at beginning.
            prev_ok = (prev_row is None) or (lc(prev_row['end location']) == "ehvbst")
            next_ok = True  # no special constraint on next_row here

            if not prev_ok or not next_ok:
                continue

            # check time gaps with routing (prev->trip.start and trip.end->next)
            # left side
            if prev_row is None:
                need_L = 0.0; segs_L = []
            else:
                if lc(prev_row['end location']) == lc(trip_sloc):
                    need_L, segs_L = TURNOVER, []
                else:
                    total_L, segs_L = route_time(tt, prev_row['end location'], trip_sloc)
                    if total_L is None:
                        continue
                    need_L = total_L + TURNOVER
                if gap_min(prev_row['_end'], trip_start) < need_L:
                    continue

            # right side
            if next_row is None:
                need_R = 0.0; segs_R = []
            else:
                if lc(trip_eloc) == lc(next_row['start location']):
                    need_R, segs_R = TURNOVER, []
                else:
                    total_R, segs_R = route_time(tt, trip_eloc, next_row['start location'])
                    if total_R is None:
                        continue
                    need_R = total_R + TURNOVER
                if gap_min(trip_end, next_row['_start']) < need_R:
                    continue

            # ok: move
            plan.loc[trip_idx, 'bus'] = tgt_bus
            base = plan.loc[trip_idx].copy()

            # material before (prev -> trip.start)
            if prev_row is not None and segs_L:
                t0 = prev_row['_end']
                for (a,b,m) in segs_L:
                    if m <= 0: continue
                    mat = base.copy()
                    mat['activity'] = 'material trip'
                    mat['line'] = np.nan
                    mat['start location'] = a
                    mat['end location']   = b
                    mat['_start'] = t0
                    mat['_end']   = t0 + pd.Timedelta(minutes=m)
                    mat['energy consumption'] = energy_material(m)
                    plan = pd.concat([plan, pd.DataFrame([mat])], ignore_index=True)
                    t0 = mat['_end']

            # material after (trip.end -> next.start)
            if next_row is not None and segs_R:
                t1 = base['_end']
                for (a,b,m) in segs_R:
                    if m <= 0: continue
                    mat2 = base.copy()
                    mat2['activity'] = 'material trip'
                    mat2['line'] = np.nan
                    mat2['start location'] = a
                    mat2['end location']   = b
                    mat2['_start'] = t1
                    mat2['_end']   = t1 + pd.Timedelta(minutes=m)
                    mat2['energy consumption'] = energy_material(m)
                    plan = pd.concat([plan, pd.DataFrame([mat2])], ignore_index=True)
                    t1 = mat2['_end']

            placed = True
            moves_done += 1
            print(f"✔ moved bad trip idx={trip_idx} from bus {src_bus} -> bus {tgt_bus} (slot={tag})")
            break

        if placed:
            break

    if not placed:
        print(f"⚠ could not relocate trip idx={trip_idx}. Consider relaxing TURNOVER or moving it manually.")

# ---------- write output in original format ----------
out = plan.copy()
out['start time'] = pd.to_datetime(out['_start']).dt.strftime("%H:%M:%S")
out['end time']   = pd.to_datetime(out['_end']).dt.strftime("%H:%M:%S")
out = out.drop(columns=['_start','_end'], errors='ignore')
out.to_excel(OUT_FILE, sheet_name="Sheet1", index=False)
print(f"\n✅ patched and saved to: {OUT_FILE}")
print(f"🧩 trips moved: {moves_done}")


found 2 'ehvapt→ehvbst' missing-timetable transitions.
⚠ could not relocate trip idx=40. Consider relaxing TURNOVER or moving it manually.
⚠ could not relocate trip idx=41. Consider relaxing TURNOVER or moving it manually.

✅ patched and saved to: busplan_patch_missing_timetable_fixed.xlsx
🧩 trips moved: 0
